# Fine Tuning Model Open AI

Notebook ini didasarkan pada panduan terkini yang disediakan dalam dokumentasi [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) dari Open AI.

Fine-tuning meningkatkan kinerja model dasar untuk aplikasi Anda dengan melatih ulang menggunakan data tambahan dan konteks yang relevan dengan kasus penggunaan atau skenario tertentu tersebut. Perlu dicatat bahwa teknik rekayasa prompt seperti _few shot learning_ dan _retrieval augmented generation_ memungkinkan Anda meningkatkan prompt default dengan data relevan untuk meningkatkan kualitas. Namun, pendekatan ini dibatasi oleh ukuran jendela token maksimum dari model dasar yang ditargetkan.

Dengan fine-tuning, kita secara efektif melatih ulang model itu sendiri dengan data yang diperlukan (memungkinkan kita menggunakan lebih banyak contoh daripada yang dapat muat dalam jendela token maksimum) - dan menerapkan versi _custom_ dari model yang tidak lagi perlu diberikan contoh saat waktu inferensi. Ini tidak hanya meningkatkan efektivitas desain prompt kita (kita memiliki lebih banyak fleksibilitas dalam menggunakan jendela token untuk hal lain) tetapi juga berpotensi meningkatkan biaya kita (dengan mengurangi jumlah token yang perlu kita kirim ke model saat waktu inferensi).

Fine tuning memiliki 4 langkah:
1. Siapkan data pelatihan dan unggah.
1. Jalankan pekerjaan pelatihan untuk mendapatkan model yang telah di-fine-tune.
1. Evaluasi model yang telah di-fine-tune dan iterasi untuk kualitas.
1. Terapkan model yang telah di-fine-tune untuk inferensi saat sudah puas.

Perlu dicatat bahwa tidak semua model dasar mendukung fine-tuning - [periksa dokumentasi OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) untuk informasi terbaru. Anda juga dapat melakukan fine-tune pada model yang sebelumnya sudah di-fine-tune. Dalam tutorial ini, kita akan menggunakan `gpt-35-turbo` sebagai model dasar target untuk fine-tuning.

---


### Langkah 1.1: Siapkan Dataset Anda

Mari kita buat chatbot yang membantu Anda memahami tabel periodik unsur dengan menjawab pertanyaan tentang suatu unsur dengan sebuah limerik. Dalam tutorial sederhana _ini_, kita hanya akan membuat dataset untuk melatih model dengan beberapa contoh respons yang menunjukkan format data yang diharapkan. Dalam kasus penggunaan dunia nyata, Anda perlu membuat dataset dengan banyak contoh lebih banyak. Anda juga mungkin dapat menggunakan dataset terbuka (untuk domain aplikasi Anda) jika ada, dan memformat ulang untuk digunakan dalam fine-tuning.

Karena kita fokus pada `gpt-35-turbo` dan mencari respons satu putaran (chat completion) kita dapat membuat contoh menggunakan [format yang disarankan ini](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) yang mencerminkan persyaratan chat completion OpenAI. Jika Anda mengharapkan konten percakapan multi-putaran, Anda akan menggunakan [format contoh multi-putaran](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) yang mencakup parameter `weight` untuk menandakan pesan mana yang harus digunakan (atau tidak) dalam proses fine-tuning.

Kita akan menggunakan format satu putaran yang lebih sederhana untuk tutorial kita di sini. Data dalam format [jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) dengan 1 catatan per baris, masing-masing direpresentasikan sebagai objek berformat JSON. Cuplikan di bawah menunjukkan 2 catatan sebagai contoh - lihat [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) untuk set contoh lengkap (10 contoh) yang akan kita gunakan untuk tutorial fine-tuning kita. **Catatan:** Setiap catatan _harus_ didefinisikan dalam satu baris (tidak dipisah ke beberapa baris seperti biasanya dalam file JSON yang diformat)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Dalam kasus penggunaan dunia nyata Anda akan membutuhkan set contoh yang jauh lebih besar untuk hasil yang baik - pertukaran akan terjadi antara kualitas respons dan waktu/biaya untuk fine-tuning. Kita menggunakan set kecil agar dapat menyelesaikan fine-tuning dengan cepat untuk mengilustrasikan prosesnya. Lihat [contoh OpenAI Cookbook ini](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) untuk tutorial fine-tuning yang lebih kompleks.


---

### Langkah 1.2 Unggah Dataset Anda

Unggah data menggunakan Files API [seperti yang dijelaskan di sini](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Perlu dicatat bahwa untuk menjalankan kode ini, Anda harus terlebih dahulu melakukan langkah-langkah berikut:
 - Menginstal paket Python `openai` (pastikan Anda menggunakan versi >=0.28.0 untuk fitur terbaru)
 - Mengatur variabel lingkungan `OPENAI_API_KEY` ke kunci API OpenAI Anda
Untuk mempelajari lebih lanjut, lihat [Panduan Pengaturan](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) yang disediakan untuk kursus ini.

Sekarang, jalankan kode untuk membuat file yang akan diunggah dari file JSONL lokal Anda.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Langkah 2.1: Buat pekerjaan Fine-tuning dengan SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Langkah 2.2: Periksa Status Pekerjaan

Berikut beberapa hal yang dapat Anda lakukan dengan API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Daftar n pekerjaan fine-tuning terakhir
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Dapatkan detail dari pekerjaan fine-tuning tertentu
- `client.fine_tuning.jobs.cancel(<job_id>)` - Batalkan pekerjaan fine-tuning
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Daftar hingga n event dari pekerjaan tersebut
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Langkah pertama dari proses ini adalah _memvalidasi file pelatihan_ untuk memastikan data dalam format yang benar.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Langkah 2.3: Lacak peristiwa untuk memantau kemajuan


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Langkah 2.4: Lihat status di Dasbor OpenAI


Anda juga dapat melihat status dengan mengunjungi situs web OpenAI dan menjelajahi bagian _Fine-tuning_ dari platform. Ini akan menunjukkan status pekerjaan saat ini, dan juga memungkinkan Anda melacak riwayat pelaksanaan pekerjaan sebelumnya. Dalam tangkapan layar ini, Anda dapat melihat bahwa pelaksanaan sebelumnya gagal, dan pelaksanaan kedua berhasil. Sebagai konteks, ini terjadi ketika pelaksanaan pertama menggunakan file JSON dengan catatan yang diformat secara tidak benar - setelah diperbaiki, pelaksanaan kedua selesai dengan sukses dan membuat model tersedia untuk digunakan.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.id.png)


Anda juga dapat melihat pesan status dan metrik dengan menggulir ke bawah lebih jauh di dasbor visual seperti yang ditunjukkan:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.id.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.id.png)|


---

### Langkah 3.1: Ambil ID & Uji Model Fine-Tuned dalam Kode


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Langkah 3.2: Muat & Uji Model Fine-Tuned di Playground

Anda sekarang dapat menguji model fine-tuned dengan dua cara. Pertama, Anda dapat mengunjungi Playground dan menggunakan menu drop-down Models untuk memilih model fine-tuned baru Anda dari opsi yang terdaftar. Pilihan lainnya adalah menggunakan opsi "Playground" yang ditampilkan di panel Fine-tuning (lihat tangkapan layar di atas) yang meluncurkan tampilan _perbandingan_ berikut yang menunjukkan versi model dasar dan model fine-tuned berdampingan untuk evaluasi cepat.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.id.png)

Cukup isi konteks sistem yang digunakan dalam data pelatihan Anda dan berikan pertanyaan uji Anda. Anda akan melihat bahwa kedua sisi diperbarui dengan konteks dan pertanyaan yang identik. Jalankan perbandingan dan Anda akan melihat perbedaan output di antara keduanya. _Perhatikan bagaimana model fine-tuned menghasilkan respons dalam format yang Anda berikan dalam contoh Anda sementara model dasar hanya mengikuti prompt sistem_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.id.png)

Anda akan melihat bahwa perbandingan juga menyediakan jumlah token untuk setiap model, dan waktu yang dibutuhkan untuk inferensi. **Contoh spesifik ini adalah contoh sederhana yang dimaksudkan untuk menunjukkan proses tetapi tidak benar-benar mencerminkan dataset atau skenario dunia nyata**. Anda mungkin memperhatikan bahwa kedua sampel menunjukkan jumlah token yang sama (konteks sistem dan prompt pengguna identik) dengan model fine-tuned membutuhkan waktu lebih lama untuk inferensi (model kustom).

Dalam skenario dunia nyata, Anda tidak akan menggunakan contoh main-main seperti ini, tetapi fine-tuning terhadap data nyata (misalnya, katalog produk untuk layanan pelanggan) di mana kualitas respons akan jauh lebih nyata. Dalam _konteks_ itu, mendapatkan kualitas respons yang setara dengan model dasar akan memerlukan rekayasa prompt kustom yang lebih banyak yang akan meningkatkan penggunaan token dan berpotensi waktu pemrosesan terkait untuk inferensi. _Untuk mencoba ini, lihat contoh fine-tuning di OpenAI Cookbook untuk memulai._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk mencapai akurasi, harap diingat bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang sahih. Untuk informasi penting, disarankan menggunakan terjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
